# 04: Bounded Contexts — Multi-Instance Kind Definitions

This notebook defines a `CleanArchitectureBoundedContext` Kind with three layers, then declares two instances of it — `payments` and `orders` — to demonstrate multi-instance patterns and explore current limitations.

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup

In [ ]:
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; output: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  const output = (new TextDecoder().decode(stdout) + new TextDecoder().decode(stderr)).trim();
  if (output) console.log(output);
  console.log(`\nExit code: ${code}`);
  return { code, output };
}

async function tree(dir: string): Promise<void> {
  const cmd = new Deno.Command("find", {
    args: [dir, "-type", "f"],
    stdout: "piped",
  });
  const { stdout } = await cmd.output();
  const files = new TextDecoder().decode(stdout).trim().split("\n")
    .map(f => f.replace(dir + "/", ""))
    .sort();
  console.log(files.join("\n"));
}

console.log("CLI path:", KSC);

---

## Step 1: Define the Four Kinds

We define:
- `DomainLayer` — pure business logic
- `ApplicationLayer` — use cases and ports
- `InfrastructureLayer` — adapters and external dependencies
- `CleanArchitectureBoundedContext` — composes all three layers

Then we declare **two instances**: `payments` and `orders`, each at a different filesystem root.

In [ ]:
const DEMO = Deno.makeTempDirSync({ prefix: "ksc-bounded-ctx-" });
console.log("Demo project:", DEMO);

// Create directory structure for both bounded contexts
for (const ctx of ["payments", "orders"]) {
  Deno.mkdirSync(`${DEMO}/src/${ctx}/domain`, { recursive: true });
  Deno.mkdirSync(`${DEMO}/src/${ctx}/application`, { recursive: true });
  Deno.mkdirSync(`${DEMO}/src/${ctx}/infrastructure`, { recursive: true });
}

// --- Payments bounded context ---

Deno.writeTextFileSync(`${DEMO}/src/payments/domain/payment.ts`, `
export interface Payment {
  id: string;
  amount: number;
  currency: string;
}

export function createPayment(amount: number, currency: string): Payment {
  return { id: Math.random().toString(36), amount, currency };
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/payments/application/process-payment.ts`, `
import { createPayment, Payment } from '../domain/payment';

export function processPayment(amount: number, currency: string): Payment {
  return createPayment(amount, currency);
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/payments/infrastructure/stripe-adapter.ts`, `
import { Payment } from '../domain/payment';

export function chargeStripe(payment: Payment): void {
  console.log('Charging', payment.amount, payment.currency);
}
`.trimStart());

// --- Orders bounded context ---

Deno.writeTextFileSync(`${DEMO}/src/orders/domain/order.ts`, `
export interface Order {
  id: string;
  items: string[];
}

export function createOrder(items: string[]): Order {
  return { id: Math.random().toString(36), items };
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/orders/application/place-order.ts`, `
import { createOrder, Order } from '../domain/order';

export function placeOrder(items: string[]): Order {
  return createOrder(items);
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/src/orders/infrastructure/order-repo.ts`, `
import { Order } from '../domain/order';

const store: Order[] = [];

export function saveOrder(order: Order): void {
  store.push(order);
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
  include: ["src/**/*.ts", "architecture.ts"],
}, null, 2));

console.log("\nProject structure:");
await tree(DEMO);

### Write `architecture.ts`

Four Kind interfaces, two `locate<T>()` instances, and one `defineContracts<T>()` call.

In [ ]:
const architectureTs = `
// --- Runtime types (inlined for self-contained demo) ---

interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}

type MemberMap<T extends Kind> = {
  [K in keyof T as K extends 'kind' | 'location' ? never : K]:
    T[K] extends Kind
      ? MemberMap<T[K]> | { path: string } & Partial<MemberMap<T[K]>> | Record<string, never>
      : never;
};
function locate<T extends Kind>(root: string, members: MemberMap<T>): MemberMap<T> {
  void root;
  return members;
}

interface ContractConfig {
  noDependency?: [string, string][];
  purity?: string[];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}

// --- Kind Definitions ---

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface ApplicationLayer extends Kind<"ApplicationLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export interface CleanArchitectureBoundedContext extends Kind<"CleanArchitectureBoundedContext"> {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}

// --- Instances ---

export const paymentsCleanArchitectureBoundedContext = locate<CleanArchitectureBoundedContext>("src/payments", {
  domain: {},
  application: {},
  infrastructure: {},
});

export const orderCleanArchitectureBoundedContext = locate<CleanArchitectureBoundedContext>("src/orders", {
  domain: {},
  application: {},
  infrastructure: {},
});

// --- Contracts ---

export const contracts = defineContracts<CleanArchitectureBoundedContext>({
  noDependency: [
    ["domain", "infrastructure"],
    ["domain", "application"],
  ],
  purity: ["domain"],
});
`.trimStart();

Deno.writeTextFileSync(`${DEMO}/architecture.ts`, architectureTs);

Deno.writeTextFileSync(`${DEMO}/kindscript.json`, JSON.stringify({
  definitions: ["architecture.ts"],
}, null, 2));

console.log("=== architecture.ts ===");
console.log(architectureTs);

---

## Step 2: Check Contracts — Clean State

Both bounded contexts have valid structure. Let's verify.

In [ ]:
await ksc("check", DEMO);

Exit code 0 — both bounded contexts pass all contracts. KindScript checked:

- **noDependency**: `domain` does not import from `infrastructure` or `application` in either context
- **purity**: `domain` has no Node.js built-in imports in either context
- **existence**: All 6 derived directories (`src/payments/domain`, `src/payments/application`, `src/payments/infrastructure`, `src/orders/domain`, `src/orders/application`, `src/orders/infrastructure`) exist

---

## Step 3: Introduce a Violation in Payments

The payments domain layer imports from its own infrastructure — a forbidden dependency. Let's see if KindScript catches it.

In [ ]:
Deno.writeTextFileSync(`${DEMO}/src/payments/domain/payment.ts`, `
import { chargeStripe } from '../infrastructure/stripe-adapter';

export interface Payment {
  id: string;
  amount: number;
  currency: string;
}

export function createPayment(amount: number, currency: string): Payment {
  const payment = { id: Math.random().toString(36), amount, currency };
  chargeStripe(payment); // BAD: domain reaching into infrastructure
  return payment;
}
`.trimStart());

console.log("Violation introduced in payments domain");

In [ ]:
await ksc("check", DEMO);

**BUG: The violation was NOT caught.** KindScript reported "All architectural contracts satisfied" with exit code 0, even though `payments/domain/payment.ts` imports from `payments/infrastructure/`.

This reveals a real limitation in the current implementation: when multiple instances share the same Kind type (`CleanArchitectureBoundedContext`), the classifier stores instance→Kind mappings in a `Map<string, ArchSymbol>` keyed by Kind type name. The second `locate()` call (orders) **overwrites** the first (payments) in this map. Contracts then bind only to the **last** instance — orders.

The payments context is effectively invisible to contract checking. We'll discuss this further in the Issues section below.

---

## Step 4: Fix and Verify

Restore the clean domain file.

In [ ]:
Deno.writeTextFileSync(`${DEMO}/src/payments/domain/payment.ts`, `
export interface Payment {
  id: string;
  amount: number;
  currency: string;
}

export function createPayment(amount: number, currency: string): Payment {
  return { id: Math.random().toString(36), amount, currency };
}
`.trimStart());

await ksc("check", DEMO);

---

## Step 5: Existence Checking — Missing Directory

What if we declare a bounded context whose directories don't exist yet?

In [ ]:
// Add a third bounded context instance pointing to directories that don't exist
const archWithThird = architectureTs + `
export const shippingCleanArchitectureBoundedContext = locate<CleanArchitectureBoundedContext>("src/shipping", {
  domain: {},
  application: {},
  infrastructure: {},
});
`;

Deno.writeTextFileSync(`${DEMO}/architecture.ts`, archWithThird);
await ksc("check", DEMO);

KindScript reports `KS70010` — the derived directories for the shipping context (`src/shipping/domain`, `src/shipping/application`, `src/shipping/infrastructure`) don't exist on disk.

You can fix this by creating the directories manually, or using `ksc scaffold --write --instance <name>`.

In [ ]:
// Scaffold requires --instance when multiple instances exist
await ksc("scaffold", "--write", "--instance", "shippingCleanArchitectureBoundedContext", DEMO);

console.log("\n--- After scaffold ---");
await ksc("check", DEMO);

---

## Issues and Design Discussion

### 1. Contract-to-Instance Binding Bug (Demonstrated Above)

**This notebook exposed a real bug.** When two instances share the same Kind type, contracts only check the *last* instance declared. The payments violation in Step 3 went undetected.

**Root cause:** In `classify-ast.service.ts`, the classifier stores instance→Kind mappings in a `Map<string, ArchSymbol>` keyed by Kind type name (line 96: `instanceSymbols.set(kindName, result.symbol)`). Since Map keys are unique, the second `locate<CleanArchitectureBoundedContext>()` overwrites the first.

**Impact:** In the multi-instance (bounded context) pattern — the primary use case for shared Kind types — only the last declared context gets contract enforcement. All earlier contexts are silently unchecked.

**Fix:** Change `instanceSymbols` from `Map<string, ArchSymbol>` to `Map<string, ArchSymbol[]>` and fan out contract binding to all instances of a given Kind type.

### 2. Scaffold Requires `--instance` for Multi-Instance

When multiple instances of the same Kind exist, `ksc scaffold` cannot disambiguate which one to scaffold. It requires `--instance <name>` to select. This is reasonable UX, but means you can't scaffold all instances in one command.

### 3. Verbose Instance Names

The variable names `paymentsCleanArchitectureBoundedContext` and `orderCleanArchitectureBoundedContext` are redundant — the type argument to `locate<CleanArchitectureBoundedContext>()` already declares the Kind. Idiomatic usage is shorter:

```typescript
export const payments = locate<CleanArchitectureBoundedContext>("src/payments", { ... });
export const orders   = locate<CleanArchitectureBoundedContext>("src/orders", { ... });
```

The variable name is just a label for the user — KindScript reads the type argument, not the variable name.

### 4. Shared vs Per-Instance Contracts

With one `defineContracts<CleanArchitectureBoundedContext>()`, the same rules apply to all instances. This is usually what you want — all clean architecture bounded contexts should enforce the same dependency rules.

If you need **different rules per context** (e.g., payments has stricter purity), you'd need to define separate Kind types:

```typescript
interface PaymentsContext extends Kind<"PaymentsContext"> { ... }
interface OrdersContext extends Kind<"OrdersContext"> { ... }
```

This is a fundamental design choice: same Kind type = same contracts.

### 5. Cross-Context Dependencies

KindScript's `noDependency` contracts operate *within* a bounded context (e.g., payments.domain cannot import from payments.infrastructure). But what about **cross-context** dependencies (payments importing from orders)?

This isn't currently expressible. You'd need a higher-level Kind that has the bounded contexts as members and defines inter-context dependency rules. This is a natural extension of the Kind system but not yet implemented.

In [ ]:
Deno.removeSync(DEMO, { recursive: true });
console.log("Demo project cleaned up.");

---

## Summary

| Concept | Status |
|---------|--------|
| **Kind definition** | Works — `interface X extends Kind<"X"> { ... }` defines the shape |
| **Multi-instance** | Works — multiple `locate<SameKind>()` at different roots |
| **Existence checking** | Works — `KS70010` fires for missing directories |
| **Scaffold** | Works — but requires `--instance` when multiple instances exist |
| **Shared contracts** | **BUG** — only the last instance gets contract enforcement |

**Key finding:** The multi-instance pattern works for existence checking (all instances are verified), but contract checking (noDependency, purity, etc.) only applies to the last declared instance. This is a bug in `classify-ast.service.ts` line 96 where a `Map<string, ArchSymbol>` should be `Map<string, ArchSymbol[]>`.